In [50]:
import json

In [51]:
with open('../../../scrapy/tutorial/test.json', 'rb') as f:
    dct = json.load(f)

In [34]:
import os
import re
import math
import pickle
import requests
import subprocess
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from loguru import logger
import librosa
import numpy as np
import glob
from scipy.io import wavfile
pd.options.display.width = 0

## Video

In [102]:
AUDIO_PARAMS = {
    'sampling_rate':16000,
    'channels':2
}
DEBUG = False
MAX_ONE_SPEAKER_UTT_LEN = 10
WINDOW_SEARCH_WIDTH = 5
NAIVE_SPLIT_TOLERANCE = 0.2

In [103]:
def process_one_video(video_url,
                      temp_video_file,
                      wav_path):
    """Download video, extract wav, delete video
    """
    
        
    ffmpeg_params = "ffmpeg -i '{}' -ac {} -ar {} -vn '{}'".format(
        temp_video_file,
        AUDIO_PARAMS['channels'],
        AUDIO_PARAMS['sampling_rate'],
        wav_path)
    
    if DEBUG:
        print(ffmpeg_params)
    
    if True:
        download_progress(video_url,
                          temp_video_file)    
        
    stdout = subprocess.Popen(ffmpeg_params,
                              shell=True,
                              stdout=subprocess.PIPE).stdout.read()
    
    msg = 'Decoding url {} stdout \n {}'.format(video_url,
                                                str(stdout))    
    logger.info(msg)

    if os.path.isfile(temp_video_file):
        os.remove(temp_video_file)
    else:
        logger.warn('File {} not found for deletion'.format(temp_video_file))
    return wav_path


def download_progress(download_url,
                      target_path,
                      disable_tqdm=False):

    r = requests.get(download_url,
                     stream=True)

    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024 * 1024
    wrote = 0

    if r.status_code == 200:
        with open(target_path, 'wb') as f:
            for data in tqdm(r.iter_content(block_size),
                             total=math.ceil(total_size//block_size),
                             unit='MB', unit_scale=True,
                             disable=disable_tqdm):
                wrote = wrote + len(data)
                f.write(data)
        if total_size != 0 and wrote != total_size:
            print("ERROR, something went wrong")                          
    else:
        msg = "Url {} responded with non 200 status".format(url)
        logger.error(msg)
        raise Exception(msg)

In [115]:
url = 'https://duma.mos.ru/uploads/conference_video/44e5c26a827e4b64154a18fd516e838efe05f005.mp4'

In [116]:
process_one_video(url, 'test_video', 'test.wav')

177MB [00:16, 11.5MB/s]                          
2019-07-26 08:26:45.229 | INFO     | __main__:process_one_video:27 - Decoding url https://duma.mos.ru/uploads/conference_video/44e5c26a827e4b64154a18fd516e838efe05f005.mp4 stdout 
 b''


'test.wav'

In [31]:
videos = []
for small_dict in dct:
    if 'video' in small_dict:
        videos.append(small_dict)

In [40]:
video_df = pd.DataFrame(videos)
video_df['hash'] = video_df['video'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [44]:
video_df['wav_path'] = 'mosduma_' + video_df['page'].astype(str) + '_' + \
                        video_df['block'].astype(str) + '_' + video_df['hash'] + '.wav'

In [ ]:
video_df.to_feather('video_df.feather')

In [46]:
video_df.head(1)

,block,page,video,hash,wav_path
0,10,1,/uploads/conference_video/f7a480de76d23c5733e4...,f7a480de76d23c5733e428153d43aab2d86ea9f4,mosduma_1_10_f7a480de76d23c5733e428153d43aab2d...


In [37]:
video_df['video'][0]

'/uploads/conference_video/f7a480de76d23c5733e428153d43aab2d86ea9f4.mp4'

In [ ]:
video_urls = list(video_df['video'].values)
wav_file_paths = list(video_df['wav_path'].values)

In [ ]:
for (video_url,
     wav_file_path) in tqdm(zip(video_urls,
                                wav_file_paths), total=len(video_urls)):
    try:
        process_one_video('https://duma.mos.ru/' + video_url,
                          'temp_video/temp_video_file.mp4',
                          'wav_folder/' + wav_file_path)
    except Exception as e:
        msg = 'Video {} caused error {}'.format(video_url,str(e))    
        logger.error(msg)
        
msg = 'Done'    
logger.info(msg)     

In [134]:
video_df = pd.read_feather('video_df.feather')

In [135]:
video_df_bin = pd.read_feather('video_df_bin.feather')

In [141]:
len(video_df)

250

In [142]:
video_df = video_df[~video_df['hash'].isin(video_df_bin['hash'].values)]

In [143]:
video_df_final = pd.concat([video_df, video_df_bin]).reset_index(drop=True)

## Text

In [173]:
text_dct = [i for i in dct if 'urls' in i]

In [174]:
text_dicts = []
for page in text_dct:
    for i, url in enumerate(page['urls']):
        text_dicts.append({'text_url':url,
                           'page':page['page'],
                           'block':i+1})

In [175]:
text_df = pd.DataFrame(text_dicts)
text_df.head(3)

,block,page,text_url
0,1,1,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...
1,2,1,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...
2,3,1,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...


In [176]:
video_df = pd.read_feather('video_df.feather')
video_df.head(3)

,block,page,video,hash,wav_path
0,10,1,/uploads/conference_video/f7a480de76d23c5733e4...,f7a480de76d23c5733e428153d43aab2d86ea9f4,mosduma_1_10_f7a480de76d23c5733e428153d43aab2d...
1,6,1,/uploads/conference_video/97244770a1a14a942175...,97244770a1a14a9421754068f867c49444ecc2f0,mosduma_1_6_97244770a1a14a9421754068f867c49444...
2,9,1,/uploads/conference_video/c7fda223c218e2dff368...,c7fda223c218e2dff3688c8cef92857f13c6a78f,mosduma_1_9_c7fda223c218e2dff3688c8cef92857f13...


In [188]:
text_video = pd.merge(video_df, text_df,  how='inner', left_on=['page','block'], right_on = ['page','block'])

In [189]:
text_video['text_path'] = 'mosduma_' + text_video['page'].astype(str) + '_' + text_video['block'].astype(str) + '.rtf'

In [190]:
len(text_video)

249

In [191]:
text_video.head(3)

,block,page,video,hash,wav_path,text_url,text_path
0,10,1,/uploads/conference_video/f7a480de76d23c5733e4...,f7a480de76d23c5733e428153d43aab2d86ea9f4,mosduma_1_10_f7a480de76d23c5733e428153d43aab2d...,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...,mosduma_1_10.rtf
1,6,1,/uploads/conference_video/97244770a1a14a942175...,97244770a1a14a9421754068f867c49444ecc2f0,mosduma_1_6_97244770a1a14a9421754068f867c49444...,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...,mosduma_1_6.rtf
2,9,1,/uploads/conference_video/c7fda223c218e2dff368...,c7fda223c218e2dff3688c8cef92857f13c6a78f,mosduma_1_9_c7fda223c218e2dff3688c8cef92857f13...,https://api.duma.mos.ru/isszd/fx/mitkfiledownl...,mosduma_1_9.rtf


In [344]:
down_wavs = list(map(lambda x: x.split('/')[1], glob.glob('wav_folder/*.wav')))

In [350]:
text_video = text_video[text_video['wav_path'].isin(down_wavs)]

In [356]:
text_paths = text_video['text_path'].values

In [359]:
text_video.reset_index(drop=True).to_feather('text_video.feather')

In [360]:
text_video.head(3)

,block,page,video,hash,wav_path,text_url,text_path
0,10,1,/uploads/conference_video/f7a480de76d23c5733e428153d43aab2d86ea9f4.mp4,f7a480de76d23c5733e428153d43aab2d86ea9f4,mosduma_1_10_f7a480de76d23c5733e428153d43aab2d86ea9f4.wav,https://api.duma.mos.ru/isszd/fx/mitkfiledownloadservlet?fileUUID=dbfilefs002080000mff3cev6pnt41h0&time=1563989069343,mosduma_1_10.rtf
1,6,1,/uploads/conference_video/97244770a1a14a9421754068f867c49444ecc2f0.mp4,97244770a1a14a9421754068f867c49444ecc2f0,mosduma_1_6_97244770a1a14a9421754068f867c49444ecc2f0.wav,https://api.duma.mos.ru/isszd/fx/mitkfiledownloadservlet?fileUUID=dbfilefs002080000mie8g4lhrv48ng0&time=1563989069343,mosduma_1_6.rtf
2,9,1,/uploads/conference_video/c7fda223c218e2dff3688c8cef92857f13c6a78f.mp4,c7fda223c218e2dff3688c8cef92857f13c6a78f,mosduma_1_9_c7fda223c218e2dff3688c8cef92857f13c6a78f.wav,https://api.duma.mos.ru/isszd/fx/mitkfiledownloadservlet?fileUUID=dbfilefs002080000mfqmp0jgco7d6qs&time=1563989069343,mosduma_1_9.rtf


## Speaker and speech

In [102]:
bad_txts = []
txts = glob.glob('text_folder/txt/*txt')
for txt in tqdm(txts):
    with open(txt, 'r') as f:
        file = f.readlines()
        
    #txt = txt.split('/')[-1]
    
    text = ''.join(file)
    if 'Председатель Московской городской Думы' not in text:
        bad_txts.append(txt)

100%|██████████| 204/204 [00:00<00:00, 793.11it/s]


In [101]:
txts[0]

'text_folder/txt/mosduma_11_6.txt'

In [93]:
txts = glob.glob('text_folder/txt/*txt')
with open(txts[0], 'r') as f:
        file = f.readlines()

In [94]:
text = ''.join(file)

In [95]:
text.split('Председатель Московской городской Думы')[1]

' Платонов В.М. \n\n\nУтреннее заседание № 1411\n\n\n                       Звучит гимн города Москвы.\n\nМинутой молчания депутаты почтили память погибших в террористических актах в Волгограде и других регионах России.\n\n\nI. СЛУШАЛИ: Проект постановления Московской городской Думы «О плане работы Московской городской Думы на I квартал 2014 года».\n \nПроект постановления № 08-13-9491/13 внесен комиссией по организации работы Думы 18 декабря 2013 года.\n\nРедактор проекта постановления: Метельский А.Н.\n\nПервое чтение.\n\nС докладом: Метельский А.Н.\n\nВопросов не поступило. \n\nВыступлений не состоялось.\n\nВ заключительном слове редактор проекта постановления – Метельский А.Н. – внес изменения в текст проекта документа. \n\nПо мотивам голосования о принятии проекта постановления в первом чтении выступлений не состоялось. \n \nГолосованием (за – 26, против – 0, воздержались – 0) проект постановления с изменениями редактора принимается в первом чтении.\n\nПо ведению: Петров А.В.\n\nП

In [98]:
speaker = re.search('[А-ЯЁ]{1}[а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.', text.split('Председатель Московской городской Думы')[1])

In [99]:
speaker

<re.Match object; span=(1, 14), match='Платонов В.М.'>

In [59]:
txts = ['text_folder/txt/mosduma_20_9.txt']

In [114]:
speeches = []
txts = glob.glob('text_folder/txt/*txt')
#txts = bad_txts
for txt in tqdm(txts):
    with open(txt, 'r') as f:
        file = f.readlines()
        
    txt = txt.split('/')[-1]
    
    text = ''.join(file)
    try:
        chairman = re.search('[А-ЯЁ]{1}[а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.', text.split('Председатель Московской городской Думы')[1]).group().strip().upper()
    except:
        chairman = 'КИРИЛЛ'
    
    if 'С Т Е Н О Г Р А М М А' in text:
        sten = text.split('С Т Е Н О Г Р А М М А')[1]
        prev_speaker = None
    elif 'СТЕНОГРАММА' in text:
        sten = text.split('СТЕНОГРАММА')[1]
        prev_speaker = None
    elif 'ПРЕДСЕДАТЕЛЬ ДУМЫ:' in text:
        sten = text.split('ПРЕДСЕДАТЕЛЬ ДУМЫ:')[1]
        prev_speaker = 'ПРЕДСЕДАТЕЛЬ ДУМЫ'
    else:
        print('OSHIBKA  ', txt)
    sten = re.sub('\n{2,}', '***', sten)
    paragraphs = sten.split('***')
    
    for paragraph in paragraphs:
        speaker_speech = paragraph.split(':')
        if 'ПРЕДСЕДАТЕЛЬСТВУЮЩИЙ НА ЗАСЕДАНИИ' in speaker_speech[0]:
            extract_speaker = speaker_speech[0].replace('ПРЕДСЕДАТЕЛЬСТВУЮЩИЙ НА ЗАСЕДАНИИ', '')
        else:
            extract_speaker = speaker_speech[0]
        speaker = re.search('[А-ЯЁ\. ]{4,}', extract_speaker)
        if (len(speaker_speech) > 1) & bool(speaker):
            speaker = speaker.group().strip().upper()
            if speaker == 'ПРЕДСЕДАТЕЛЬ ДУМЫ':
                speaker = chairman
            for sentence in (':').join(speaker_speech[1:]).split('\n'):
                if sentence.strip():
                    speeches.append({'speaker':speaker,
                                 'text':sentence,
                                 'text_path':txt})
            prev_speaker = speaker
        else:
            if prev_speaker:
                for sentence in (':').join(speaker_speech).split('\n'):
                    sentence = sentence.strip()
                    if sentence:
                        if not ((sentence[0] == '(') & (sentence[-1] == ')')):
                            speeches.append({'speaker':prev_speaker,
                                         'text':sentence,
                                         'text_path':txt})

100%|██████████| 204/204 [00:00<00:00, 210.70it/s]


In [115]:
speeches_df = pd.DataFrame(speeches)

## Merge everythong

In [117]:
text_video = pd.read_feather('text_video.feather')

In [118]:
gb = text_video.groupby(['text_path'])

In [119]:
# На один текстовый файл может быть несколько аудио - соединяем
final_list = []
for group in tqdm(gb):
    arrays = []
    for wav_path in group[1]['wav_path'].values:
        sr, wav = wavfile.read('wav_folder/' + wav_path)
        arrays.append(wav)
    merged_wav = np.concatenate(arrays)
    name = 'wav_folder/merged_wavs/' + group[0].replace('rtf', 'wav')
    wavfile.write(name, 16000, merged_wav)
    final_list.append({'wav_path':name,
                       'text_path':group[0]})

100%|██████████| 161/161 [07:57<00:00,  4.63s/it]


In [120]:
final_df = pd.DataFrame(final_list)

In [121]:
final_df.head(2)

,text_path,wav_path
0,mosduma_10_1.rtf,wav_folder/merged_wavs/mosduma_10_1.wav
1,mosduma_10_10.rtf,wav_folder/merged_wavs/mosduma_10_10.wav


In [122]:
speeches_df['text_path'] = speeches_df['text_path'].apply(lambda x: x.replace('txt', 'rtf'))

In [123]:
text_wav_to_notmalize = pd.merge(speeches_df, final_df, on='text_path').drop_duplicates().reset_index(drop=True)

In [125]:
text_wav_to_notmalize.to_feather('text_wav_to_normalize.feather')

## Normalize in text-normalization

In [129]:
pd.read_feather('text_wav.feather')

,speaker,text,text_path,wav_path,normalized
0,ПЛАТОНОВ В.М.,"Уважаемые коллеги, 10 часов ровно. Сегодня сре...",mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,уважаемые коллеги десять часов ровно сегодня с...
1,ПЛАТОНОВ В.М.,"Я открываю утреннее, 1411-е заседание Московск...",mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,я открываю утреннее тысяча четыреста одиннадца...
2,ПЛАТОНОВ В.М.,Начинается оно с исполнения гимна города Москвы.,mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,начинается оно с исполнения гимна города москвы
3,ПЛАТОНОВ В.М.,Спасибо.,mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,спасибо
4,ПЛАТОНОВ В.М.,"Уважаемые коллеги, я прошу еще минуту вашего в...",mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,уважаемые коллеги я прошу еще минуту вашего вн...
5,ПЛАТОНОВ В.М.,С момента нашего последнего заседания в стране...,mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,с момента нашего последнего заседания в стране...
6,ПЛАТОНОВ В.М.,Со стороны Москвы все необходимое было сделано...,mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,со стороны москвы все необходимое было сделано...
7,ПЛАТОНОВ В.М.,Я прошу вас почтить память погибших минутой мо...,mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,я прошу вас почтить память погибших минутой мо...
8,ПЛАТОНОВ В.М.,"Спасибо, присаживайтесь, пожалуйста.",mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,спасибо присаживайтесь пожалуйста
9,ПЛАТОНОВ В.М.,"Уважаемые коллеги, будут ли возражения по ра...",mosduma_11_6.rtf,wav_folder/merged_wavs/mosduma_11_6.wav,уважаемые коллеги будут ли возражения по рассм...
